In [2]:
import requests

API_KEY = "1d48b5e24b27cd111582c21dcff9b8f5"
BASE_URL = "https://api.themoviedb.org/3"

def get_json(url, params=None):
    """A function to make get requests and return a json file"""

    params = params or {}
    params['api_key'] = API_KEY

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Lève une erreur si erreur API
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erreur API : {e}")
        return None 

def get_tmdb_id(imdb_id):
    """A funtion to get TMBD id from IMDb id"""

    json_file = get_json(f"{BASE_URL}/find/{imdb_id}", {"external_source" : "imdb_id"})

    if json_file == None : # Return None if API Error
        return None
    elif len(json_file['movie_results']) == 0 : # Return 0 if not in TMDB database
        return 0
    else :
        return json_file['movie_results'][0]['id']

def get_movie_details(movie_id):
    """A function to get detailed movie info by movie ID"""

    details = get_json(f"{BASE_URL}/movie/{movie_id}", {"append_to_response": "credits"})

    if details != None : 

        return {
            "tmdb_id" : movie_id,
            "imdb_id" : details.get("imdb_id"),
            "original_language" : details.get("original_language"),
            "popularity" : details.get("popularity"),
            "overview" : details.get("overview"), 
            "budget" : details.get("budget"),
            "country": details.get("origin_country"),
            "production companies" : [c["name"] for c in details.get("production_companies", [])],
            "revenue" : details.get("revenue"),
            "vote_average" : details.get("vote_average"),
            "vote_count" : details.get("vote_count"),
            "cast": [(c["name"],c["gender"],c["popularity"]) for c in details["credits"].get("cast",[])[:10]]  # main 10 actors
        }
    else : # If API Error
        return None 

In [17]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

def fetch_full_movie_data(imdb_id):
    """Fonction maître"""
    
    tmdb_id = get_tmdb_id(imdb_id)

    if tmdb_id == None :
        print(f"Échec de la récupération pour {imdb_id}")
        return None
    elif tmdb_id == 0 :
        print(f"Le film n'existe pas dans la database TMBD pour {imdb_id}")
        return 0
    else :
        details = get_movie_details(tmdb_id)
        return details

imdb_id_list = ["tt0133093", "tt0137523", "tt0068646", "tt_ID_FOIREUX"] 

results_list = []
fails_list = []
notfound_list = []

# max_workers=10 signifie que 10 requêtes tourneront en parallèle.
with ThreadPoolExecutor(max_workers=10) as executor:
    # executor.map applique la fonction fetch_full_movie_data
    # à chaque item de imdb_id_list.
    # Il maintient l'ordre et gère la file d'attente.

    results = executor.map(fetch_full_movie_data, imdb_id_list)

    results_with_ids = zip(imdb_id_list, results)

for imdb_id, data in results_with_ids:
    if data is None:
        # Échec API
        fails_list.append(imdb_id)
    elif data == 0:
        # Film non trouvé dans TMDB
        notfound_list.append(imdb_id)
    else:
        # Succès !
        results_list.append(data)

df = pd.DataFrame(results_list)

df.head()

Le film n'existe pas dans la database TMBD pour tt_ID_FOIREUX


,tmdb_id,imdb_id,original_language,popularity,overview,budget,country,production companies,revenue,vote_average,vote_count,cast
0,603,tt0133093,en,17.4289,"Set in the 22nd century, The Matrix tells the ...",63000000,[US],"[Village Roadshow Pictures, Groucho II Film Pa...",463517383,8.200,26959,"[(Keanu Reeves, 2, 8.5921), (Laurence Fishburn..."
1,550,tt0137523,en,15.7783,A ticking-time-bomb insomniac and a slippery s...,63000000,[US],"[Fox 2000 Pictures, Regency Enterprises, Linso...",100853753,8.438,30948,"[(Edward Norton, 2, 5.267), (Brad Pitt, 2, 6.8..."
2,238,tt0068646,en,27.0655,"Spanning the years 1945 to 1955, a chronicle o...",6000000,[US],"[Paramount Pictures, Alfran Productions]",245066411,8.700,22022,"[(Marlon Brando, 2, 1.6692), (Al Pacino, 2, 3...."


In [13]:
url_TMDB = 'https://api.themoviedb.org/3/find/tt1375666?external_source=imdb_id'